In [56]:
#importing required packages
import pandas as pd
import requests
import os


In [57]:
# Create 'data' folder if not exists
os.makedirs("data", exist_ok=True)

# Base API URL
url_use = "https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/{dataset_id}/exports/json"

# List of dataset IDs (2018–2023)
DATASET_IDS = [
    "social-indicators-for-city-of-melbourne-residents-2023",
    "social-indicators-for-city-of-melbourne-residents-2022",
    "social-indicators-for-city-of-melbourne-residents-2021",
    "social-indicators-for-city-of-melbourne-residents-2020",
    "social-indicators-for-city-of-melbourne-residents-2019",
    "social-indicators-for-city-of-melbourne-residents-2018"
]

all_years_data = []


In [58]:
# Step 2: Fetch and Filter Food Security Data
for dataset_id in DATASET_IDS:
    print(f"Fetching data for {dataset_id}...")
    dataset_url = url_use.format(dataset_id=dataset_id)
    response = requests.get(dataset_url)

    if response.status_code != 200:
        print(f"Error fetching {dataset_id}: HTTP {response.status_code}")
        continue

    data = response.json()
    if not data:
        print(f"No data found for {dataset_id}")
        continue

    df = pd.DataFrame(data)
    if "topic" in df.columns:
        df = df[df["topic"].str.contains("Food security", na=False)]

    all_years_data.extend(df.to_dict("records"))
    print(f"Fetched {len(df)} records.")


Fetching data for social-indicators-for-city-of-melbourne-residents-2023...
Fetched 90 records.
Fetching data for social-indicators-for-city-of-melbourne-residents-2022...
Fetched 90 records.
Fetching data for social-indicators-for-city-of-melbourne-residents-2021...
Fetched 72 records.
Fetching data for social-indicators-for-city-of-melbourne-residents-2020...
Fetched 72 records.
Fetching data for social-indicators-for-city-of-melbourne-residents-2019...
Fetched 72 records.
Fetching data for social-indicators-for-city-of-melbourne-residents-2018...
Fetched 72 records.


In [59]:
# Step 3: Combine and Save Initial Dataset
all_df = pd.DataFrame(all_years_data)

# Save first raw fetch
all_df.to_csv("data/food_security_all_years.csv", index=False)
print("\nCombined raw data saved to 'data/food_security_all_years.csv'.")

# Show preview
display(all_df.head())



Combined raw data saved to 'data/food_security_all_years.csv'.


,indicator,type,topic,description,response,respondent_group,year,sample_size,result,format,respondent_group0
0,6,Council Plan Indicator,Food security,Experienced food insecurity (worried food woul...,"Yes, in the last 12 months",18-24 years,2023,274,51.7,Per cent,NaN
1,6,Council Plan Indicator,Food security,Experienced food insecurity (worried food woul...,"Yes, in the last 12 months",35-44 years,2023,229,27.4,Per cent,NaN
2,6,Council Plan Indicator,Food security,Experienced food insecurity (worried food woul...,"Yes, in the last 12 months",45-54 years,2023,128,15.9,Per cent,NaN
3,6,Council Plan Indicator,Food security,Experienced food insecurity (worried food woul...,"Yes, in the last 12 months",Female,2023,761,34.8,Per cent,NaN
4,6,Council Plan Indicator,Food security,Experienced food insecurity (worried food woul...,"Yes, in the last 12 months",City of Melbourne,2023,1376,31.7,Per cent,NaN


In [60]:
all_df.shape

(468, 11)

In [61]:
#checking missing values
missing_values = all_df.isnull().sum()
print("Missing values:\n", missing_values)

#data types
data_types = all_df.dtypes
print("Data Types:\n", data_types)

Missing values:
 indicator              0
type                   0
topic                  0
description            0
response               0
respondent_group      72
year                   0
sample_size            0
result                 0
format                 0
respondent_group0    396
dtype: int64
Data Types:
 indicator             object
type                  object
topic                 object
description           object
response              object
respondent_group      object
year                  object
sample_size            int64
result               float64
format                object
respondent_group0     object
dtype: object


* respondent_group    72
* respondent_group0    396 empty
* total record = 468
* difference = 72
* checked the fetched dataset after merging csv
* also print where all the respondent_group0 and respondent_group are empty.
* probably for those 72 records missing for respondent_group, the relevant data is in respondent_group

In [62]:
#checking where only respondent_group is null
respondent_group_null = all_df[all_df['respondent_group'].isnull() & all_df['respondent_group0'].notnull()]
print(f"Number of records where respondent_group is null but respondent_group0 is not: {len(respondent_group_null)}")
if len(respondent_group_null) > 0:
    print("Sample records where respondent_group is null but respondent_group0 is not:")
    print(respondent_group_null.head())

print("\n************************************\n")

#checking where only respondent_group0 is null
respondent_group0_null = all_df[all_df['respondent_group0'].isnull() & all_df['respondent_group'].notnull()]
print(f"Number of records where respondent_group0 is null but respondent_group is not: {len(respondent_group0_null)}")
if len(respondent_group0_null) > 0:
    print("Sample records where respondent_group0 is null but respondent_group is not:")
    print(respondent_group0_null.head())

Number of records where respondent_group is null but respondent_group0 is not: 72
Sample records where respondent_group is null but respondent_group0 is not:
    indicator                    type          topic  \
180         6  Council Plan Indicator  Food security   
181         6  Council Plan Indicator  Food security   
182         6  Council Plan Indicator  Food security   
183         6  Council Plan Indicator  Food security   
184         6  Council Plan Indicator  Food security   

                                           description  \
180  Experienced food insecurity (worried food woul...   
181  Experienced food insecurity (worried food woul...   
182  Experienced food insecurity (worried food woul...   
183  Experienced food insecurity (worried food woul...   
184  Experienced food insecurity (worried food woul...   

                       response respondent_group  year  sample_size  result  \
180  Yes, in the last 12 months              NaN  2021          416    32.5  

* from above observed result and cross checking the dataset in a `.csv` format, I can see for `year 2021` dataset, `respondent_group` column is empty (for 72 records).
* so, instead the data for those 72 records of `year 2021` is in `respondent_group0` column. And hence other than 72 records, the rest 396 (498-72) records are empty in the case of `respondent_group0`, as those are stored in `respondent_group` column.
* so now i will proceed to combine these two column in one unifined column as `respondent_group`

In [63]:
#combining respondent_group and respondent_group0 into a unified column
all_df['respondent_group'] = all_df['respondent_group'].combine_first(all_df['respondent_group0'])

#droping the respondent_group0 column as it is not needed now post combination
all_df.drop(columns=['respondent_group0'], inplace=True)

#verifying 
missing_values = all_df.isnull().sum()
print("Missing values:\n", missing_values)

#saving for comparing and verifying with the previous fetched data 
all_df.to_csv("data/food_security_corrected_all_years.csv", index=False)
print("\nAll data saved to 'food_security_corrected_all_years.csv'.")

# Preview the updated DataFrame
print(all_df.head())

Missing values:
 indicator           0
type                0
topic               0
description         0
response            0
respondent_group    0
year                0
sample_size         0
result              0
format              0
dtype: int64

All data saved to 'food_security_corrected_all_years.csv'.
  indicator                    type          topic  \
0         6  Council Plan Indicator  Food security   
1         6  Council Plan Indicator  Food security   
2         6  Council Plan Indicator  Food security   
3         6  Council Plan Indicator  Food security   
4         6  Council Plan Indicator  Food security   

                                         description  \
0  Experienced food insecurity (worried food woul...   
1  Experienced food insecurity (worried food woul...   
2  Experienced food insecurity (worried food woul...   
3  Experienced food insecurity (worried food woul...   
4  Experienced food insecurity (worried food woul...   

                     respons

In [64]:
#dropping uneccessary columns
all_df = all_df.drop(columns=['indicator', 'type', 'topic', 'response', 'format'])

print("Filtered Data Sample:\n", all_df.head())

Filtered Data Sample:
                                          description   respondent_group  year  \
0  Experienced food insecurity (worried food woul...        18-24 years  2023   
1  Experienced food insecurity (worried food woul...        35-44 years  2023   
2  Experienced food insecurity (worried food woul...        45-54 years  2023   
3  Experienced food insecurity (worried food woul...             Female  2023   
4  Experienced food insecurity (worried food woul...  City of Melbourne  2023   

   sample_size  result  
0          274    51.7  
1          229    27.4  
2          128    15.9  
3          761    34.8  
4         1376    31.7  


In [65]:
#unique values in each columns
unique_counts = {col: len(all_df[col].unique()) for col in all_df.columns}
print("\nCounts of unique values in each column:\n", unique_counts)


Counts of unique values in each column:
 {'description': 6, 'respondent_group': 21, 'year': 6, 'sample_size': 222, 'result': 266}


In [66]:
#checking for decription, year, respondent group
unique_descriptions = all_df['description'].unique()
unique_years = all_df['year'].unique()
unique_respondent_groups = all_df['respondent_group'].unique()

print("Unique values in DESCRIPTION:\n", unique_descriptions)
print("\nUnique values in YEAR:\n", unique_years)
print("\nUnique values in RESPONDENT GROUP:\n", unique_respondent_groups)

Unique values in DESCRIPTION:
 ['Experienced food insecurity (worried food would run out and/or skipped meals and/or ran out of food and/or accessed emergency food relief services)'
 'Worried food would run out' 'Skipped meals' 'Ran out of food'
 'Accessed emergency food relief services'
 'Experienced food insecurity (worried food would run out and/or skipped meals and/or ran out of food)']

Unique values in YEAR:
 ['2023' '2022' '2021' '2020' '2019' '2018']

Unique values in RESPONDENT GROUP:
 ['18-24 years' '35-44 years' '45-54 years' 'Female' 'City of Melbourne'
 'North Melbourne 3051 / West Melbourne 3003'
 'South Wharf / Southbank 3006'
 'South Yarra 3141 / Melbourne/St Kilda Road 3004' '65+ years'
 'Carlton 3053' 'Kensington / Flemington 3031' 'Melbourne 3000'
 'Parkville 3052' 'Docklands 3008' 'East Melbourne 3002' 'Male'
 '55-64 years' '25-34 years' 'Kensington/ Flemingon 3031'
 'Southbank/ South Wharf 3006'
 'South Yarra 3141 / Melbourne (St Kilda Road) 3004']


In [67]:
#replacing the inconsistent values
#Kensington/ Flemingon 3031 with Kensington / Flemington 3031
#Southbank/ South Wharf 3006 with South Wharf / Southbank 3006
#South Yarra 3141 / Melbourne (St Kilda Road) 3004 with South Yarra 3141 / Melbourne/St Kilda Road 3004

all_df['respondent_group'] = all_df['respondent_group'].replace({
    'Kensington/ Flemingon 3031': 'Kensington / Flemington 3031',
    'Southbank/ South Wharf 3006': 'South Wharf / Southbank 3006',
    'South Yarra 3141 / Melbourne (St Kilda Road) 3004': 'South Yarra 3141 / Melbourne/St Kilda Road 3004'
})

#lets verify
print("Unique values in RESPONDENT GROUP after correction:\n", all_df['respondent_group'].unique())
print("\nCount of unique values: ",len(all_df['respondent_group'].unique()))

Unique values in RESPONDENT GROUP after correction:
 ['18-24 years' '35-44 years' '45-54 years' 'Female' 'City of Melbourne'
 'North Melbourne 3051 / West Melbourne 3003'
 'South Wharf / Southbank 3006'
 'South Yarra 3141 / Melbourne/St Kilda Road 3004' '65+ years'
 'Carlton 3053' 'Kensington / Flemington 3031' 'Melbourne 3000'
 'Parkville 3052' 'Docklands 3008' 'East Melbourne 3002' 'Male'
 '55-64 years' '25-34 years']

Count of unique values:  18


In [69]:
all_df.head()

,description,respondent_group,year,sample_size,result
0,Experienced food insecurity (worried food woul...,18-24 years,2023,274,51.7
1,Experienced food insecurity (worried food woul...,35-44 years,2023,229,27.4
2,Experienced food insecurity (worried food woul...,45-54 years,2023,128,15.9
3,Experienced food insecurity (worried food woul...,Female,2023,761,34.8
4,Experienced food insecurity (worried food woul...,City of Melbourne,2023,1376,31.7


In [70]:
# Step 7: Map description to clean insecurity_type
insecurity_mapping = {
    "Ran out of food": "Ran out of food",
    "Skipped meals": "Skipped meals",
    "Worried food would run out": "Worried food would run out",
    "Experienced food insecurity (worried food would run out and/or skipped meals and/or ran out of food)": "Insecurity (multiple concerns)",
    "Experienced food insecurity (worried food would run out and/or skipped meals and/or ran out of food and/or accessed emergency food relief services)": "Insecurity (multiple concerns + relief)",
    "Accessed emergency food relief services": "Accessed food relief services"
}

# Map and create new column
all_df['insecurity_type'] = all_df['description'].map(insecurity_mapping)
print("Mapped insecurity_type successfully.")

# Preview
display(all_df[['description', 'insecurity_type']].drop_duplicates())


Mapped insecurity_type successfully.


,description,insecurity_type
0,Experienced food insecurity (worried food woul...,Insecurity (multiple concerns + relief)
8,Worried food would run out,Worried food would run out
17,Skipped meals,Skipped meals
23,Ran out of food,Ran out of food
31,Accessed emergency food relief services,Accessed food relief services
180,Experienced food insecurity (worried food woul...,Insecurity (multiple concerns)


In [71]:
#final dataset
final_path = "data/cleaned_food_security.csv"
all_df.to_csv(final_path, index=False)

print(f"Cleaned dataset saved at → {final_path}")
print(f"Final shape: {all_df.shape}")
all_df.head()


Cleaned dataset saved at → data/cleaned_food_security.csv
Final shape: (468, 6)


,description,respondent_group,year,sample_size,result,insecurity_type
0,Experienced food insecurity (worried food woul...,18-24 years,2023,274,51.7,Insecurity (multiple concerns + relief)
1,Experienced food insecurity (worried food woul...,35-44 years,2023,229,27.4,Insecurity (multiple concerns + relief)
2,Experienced food insecurity (worried food woul...,45-54 years,2023,128,15.9,Insecurity (multiple concerns + relief)
3,Experienced food insecurity (worried food woul...,Female,2023,761,34.8,Insecurity (multiple concerns + relief)
4,Experienced food insecurity (worried food woul...,City of Melbourne,2023,1376,31.7,Insecurity (multiple concerns + relief)
